#  <font color=blue> ****Predicting Customer Behavior using machine learning algorithms**** </font>

## Introduction


Customer behavior analysis is crucial for businesses to better understand their customers' needs, preferences, and buying patterns. This understanding enables marketing teams to tailor their strategies more effectively, ensuring efficient outreach, increased customer satisfaction, and enhanced marketing strategies. Machine learning and predictive analysis are essential tools for uncovering insights from vast amounts of data, helping businesses identify customer needs and behavior trends. With the growing availability of customer data from sources like online transactions, social media, and customer service interactions, traditional analysis methods are becoming insufficient. As businesses struggle to extract meaningful insights, they face missed opportunities and inefficiencies in targeting and customer engagement efforts.

## Challenges

With the overwhelming volume of customer data, businesses face challenges in effectively harnessing this information to drive strategies. Traditional analysis methods fall short when it comes to analyzing the vast datasets now available. The main challenges include:

Extracting meaningful insights from large datasets to uncover key behavioral patterns and trends.
Missed opportunities due to the inability to fully leverage data for targeted marketing and customer engagement.
Inefficiencies in resource allocation because of limited insight into customer segmentation and responsiveness.

## Objective 

 An automobile company has plans to enter new markets with their existing products and after intensive market research, they’ve realized that the behavior of the new market is like their existing market. In their existing market, the sales team has classified all customers into 4 segments (A, B, C, D). Then, they performed segmented outreach and communication for a different segment of customers. This strategy has worked exceptionally well for them. Accordingly, they plan to use the same strategy for the new markets.The objective of the project is to predict the appropriate customer segments(A,B,C,D ) in a new market based on the existing market behavior by leveraging  machine learning techniques thus hepls:

 -	Personalized marketing:   help the sales team to create highly targeted marketing campaigns focused on the interest and behaviors of specific customer groups, leading to a more satisfying customer experience
 
 -	Efficient resource allocation:  business can focus their marketing, sales, and product development effort on segments that respond positively which maximize revenue
 
 -	Increase customer retention:  by meeting the specific needs of each segment business can build relationships with customers, increasing loyalty and reducing churn 

## Dataset and Data fields :


The dataset includes information on customers in the existing market , with fields age, gender , martial status, education , profession , work experience , family size, spending score and de-idenified categories. 

## Import Necessary libraries

In [ ]:
import pandas as pd # data processing , csv file(e.g. pd.read_s-csv)
import numpy as np  # linear algebra
import seaborn as sns # data visualization
import matplotlib.pyplot as plt # data visualization 
import plotly.express as px      
import plotly.graph_objs as go  # data visualization
from plotly import tools
from plotly.subplots import make_subplots
from  sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.preprocessing  import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from  sklearn.feature_selection import mutual_info_classif
import ipywidgets as widgets
import streamlit as st


plt.figure(figsize=(15,8))
sns.set(rc={'figure.figsize':(12,6)})


## Loading and Preprocessing the data

In [ ]:
train = pd.read_csv("https://raw.githubusercontent.com/AbaditEstif/machine_learning/main/Train.csv")
test= pd.read_csv("https://raw.githubusercontent.com/AbaditEstif/machine_learning/main/Test.csv")

In [ ]:
train.shape

In [ ]:
train.duplicated().sum()

In [ ]:
test.shape

In [ ]:
test.duplicated().sum()

In [ ]:
# Concatenate both train and test datasets


data = pd.concat([train,test])

### Data understanding 

My aim is to observe and study the categorical and numerical features of the dataset. by examining the unique value counts. I can understand the distribution and quality of the data and can get insights about the quality of the data,potential issues such as duplicates , imbalances and missing values.

In [ ]:
print("Number of rows:", len(data))
data.head(10)

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
# les rename Var_1 column for better understanding 

data= data.rename(columns={"Var_1":"Category"})

In [ ]:
# Checking for misspelled or unsual values in columns


print("Gender:", data['Gender'].unique())
print("Ever_Maried:", data['Ever_Married'].unique())
print("Graduated:", data['Graduated'].unique())
print("Spending_Score:", data['Spending_Score'].unique())
print("Profession:", data['Profession'].unique())
print("Segmentation:", data['Segmentation'].unique())
print("Category:", data['Category'].unique())




### Statistical Analysis

we will use data.describe(), which will give a descriptive overview of the dataset

In [ ]:
 # describe Numerical features
    
num_data= data.describe()
num_data.style.background_gradient(cmap='Oranges')



The above table shows the count,mean standard deviation,min,25%,50%,75% and max values for each column and we can observe that:
- customer age ranges from 18 to  89 years old with mean of 43.5
- works experience ranges from 0 to 14 year with mean of 2.64
- family size ranges from 1 to 9 with  mean of 2.84


In [ ]:
data.duplicated().sum()

In [ ]:
data.columns

In [ ]:
data['Age'].value_counts( dropna= False)

In [ ]:
data['Age'].unique()

In [ ]:
print("Gender:", data['Gender'].value_counts( dropna= False)),'\n',

print("Graduated:", data['Graduated'].value_counts( dropna= False)),'\n',

print("Spending_Score:", data['Spending_Score'].value_counts( dropna= False))


In [ ]:
data['Profession'].value_counts(dropna=False)

In [ ]:
data['Work_Experience'].value_counts(dropna= False)

In [ ]:
data['Family_Size'].value_counts(dropna= False)

### Checking and Handling missing values

In [ ]:
# checking missing values

data.isnull().sum()

 ### Handling missing values

I will use imputing mechanism to fill the missing value and use mode for categorical columns and median for numerical columns.


In [ ]:
# Replacing missing values in categorical columns with mode and Numerical columns with median

data['Family_Size'].fillna(data['Family_Size'].median(),inplace=True)
data['Work_Experience'].fillna(data['Work_Experience'].median(),inplace=True)

data['Graduated'].fillna(data['Graduated'].mode()[0],inplace=True)
data['Ever_Married'].fillna(data['Ever_Married'].mode()[0],inplace=True)
data['Profession'].fillna(data['Profession'].mode()[0],inplace=True)
data['Category'].fillna(data['Category'].mode()[0],inplace=True)


In [ ]:
data.isnull().sum()

In [ ]:
data.shape

###  Checking and handling outliers

Outliers can be detected using visualizations, implementing mathematical formulas on the dataset or using statistical 
approach.

### Visualizing and removing outliers using Box plot

It captures the summary of the data with a simple box and whiskers and summarizes using 25th ,50th and 75th percentiles.
And it can help us to get insights (quartiles, median and outliers ) of the data , potential outliers and understand the
centeral tendency,

In [ ]:
# Checking outlier for the numerical features

numerical_cols=['Age','Work_Experience','Family_Size']

for column in numerical_cols:
    plt.figure(figsize=(10,6))
    sns.boxplot(x=data[column])
    plt.title(f'{column} Distribution')
    plt.show()




From the above graph we can observe that work_experience values above 10 are acting as outliers. To address those outliers I will apply IQR(Inter Quartile Range). IQR is the most commonly used and most trusted approach used in the resarch field.
- IQR= Quartile3-Quartile1, this formula  will provide a measure of the spread of the middle 50% of the data in the work_experience columns .
 
 - I am calculating the interquartile rage(IQR) for work experience column,
 - first computes the first quartile (Q1) and third quartile(Q3) using the midpoint method, then calcualte thr IQR as the difference between Q3 and Q1, providing the middle 50% of work experience.

In [ ]:
Q1 = np.percentile(data['Work_Experience'], 25 , method ='midpoint')

Q3= np.percentile(data['Work_Experience'], 75, method ='midpoint')

IQR= Q3-Q1

print(IQR)

In [ ]:
# let define the upper and lower bound (1.5*IQR)

# Above Uper bound

upper =Q3+1.5*IQR

upper_array =np.array(data['Work_Experience']>= upper)
print("Upper Bound:", upper)
print(upper_array.sum())

# Below lower bound

lower= Q1-1.5*IQR
lower_array = np.array(data['Work_Experience']<= lower)
print("Lower Bound:", lower)
print(lower_array.sum())
 
    

In [ ]:
# let remove the outlier from work experience column

data['Work_Experience']= data['Work_Experience'].apply(lambda x: lower if x<lower else(upper if x>upper else x))


In [ ]:
print(data['Work_Experience'].describe())

In [ ]:
# let examine the modified work experience after removal of the outliers

plt.figure(figsize=(10,6))
sns.boxplot(x= data['Work_Experience'])
plt.title('Box plot for Work Experience')
plt.show()

## Exploratory Data Analysis(EDA)

EDA is a crucial step in the data analysis process as it helps studying, exploring, and visualizing information to derive important insights by using statistical tools and visualizations.It aids to find patterns, trends, and relationships between the variables.

### Univariate Analysis

Univare analysis involves looking at the distribution of a single variable.It is an brilliant way to understand a dataset’s range and spread of data. I will use plotly  to create univariate plots quickly.

#### Calculate Summary Statistics For the Age of the Customer

In [ ]:
data['Age'].mean()

In [ ]:
data['Age'].median()

In [ ]:
data['Age'].std()

In [ ]:
# create frequency table for 'Age'

data['Age'].value_counts()

In [ ]:
fig= px.histogram(data, x="Age", nbins=20, title= "Age Distribution", template="presentation", text_auto=True)
fig.update_layout(bargap=0.1)
fig.show()

#### Percentage distribution of Customer segments

In [ ]:
px.pie(data, names= 'Segmentation', title ="Percentage of each segment",template="presentation")

####  Gender Distribution  

In [ ]:


gender= data.groupby(["Gender"]).size().rename("count").reset_index()

In [ ]:
gender.head()

In [ ]:
px.bar(gender, x="Gender",y="count",color="Gender",text="count", color_discrete_sequence=["gray","green"], title="Distribution of Gender ",
      template="presentation")

####  Distribution of the customer’s professional inclination 

In [ ]:
profession= data.groupby(["Profession"]).size().rename("count").reset_index()

In [ ]:
profession.head()

In [ ]:
px.bar(profession, x="Profession",y="count", color="Profession", text="count", title= " Distribution of Profession ",
      template="presentation")

In [ ]:
#fig= make_subplots(rows= 1,cols=2)
#fig.add_trace(go.bar(x=data['Graduated'].value_counts().index,y=data['Graduated'].value_counts().values),row=1,col=1)
#fig.show()

In [ ]:
 px.pie(data_frame= profession, values="count", names="Profession",color="Profession",title=" Distribution of Profession ",
       template="presentation", width= 800,height=600,hole=0.5)

#### Distribution of Marital Status 

In [ ]:
marital_status= data.groupby(["Ever_Married"]).size().rename("count").reset_index()

In [ ]:
marital_status.head()

In [ ]:
px.bar(marital_status, x= "Ever_Married",y="count",color="Ever_Married", text="count", color_discrete_sequence=["brown","gray"],
      template="presentation")

In [ ]:
px.pie(data_frame= marital_status, values="count", names="Ever_Married", color="Ever_Married", title= "Marital Status of the Customer",
      template="presentation")

#### Distribution  of Educational Level 

In [ ]:
grad= data.groupby(["Graduated"]).size().rename("count").reset_index()

In [ ]:
grad.head()

In [ ]:
px.pie(data_frame= grad,values="count", names="Graduated", color="Graduated", title= " Is the Customer Graduate ", template="presentation") 

####   Work Experience in years

In [ ]:
work= data.groupby(["Work_Experience"]).size().rename("count").reset_index()

In [ ]:
work.tail()

In [ ]:
px.bar(work, x="Work_Experience",y="count", text="count", title= "Work Experience in Years", hover_name="Work_Experience",
       color= "Work_Experience",color_discrete_sequence=["orange","red","yellow","brown","gray","blue","purple"],orientation="v",
      template="presentation")

####   Spending Score of the Customer

In [ ]:
score= data.groupby(["Spending_Score"]).size().rename("count").reset_index()

In [ ]:
score.head()

In [ ]:
pie_chart= px.pie(data_frame= score, values= 'count', names='Spending_Score',color='Spending_Score',
                  title= 'Spending Score of the Customer',width= 800,height=500,hole=0.6,template="presentation")
pie_chart.show()

**Here are some insights regarding the spendig behavior of the customers**

**Majority of Customers Are Low Spenders (60.7%)**: A significant portion of the customer base falls into the low spending category. This suggests that many customers may not be fully engaged with the products, or they may be price-sensitive. Understanding the factors driving this low engagement could help the automobile company to develop targeted marketing strategies to encourage higher spending.

**Moderate Engagement with Average Spenders (24.3%)**: About a quarter of the customers are classified as average spenders. This group might be more responsive to promotional efforts. Consider tailoring marketing campaigns that specifically target this segment to encourage them to move into the high spending category.

**Small Percentage of High Spenders (15%)**: The relatively small proportion of high spenders indicates that there is a niche audience that generates a disproportionate amount of revenue. It's important to identify the characteristics and behaviors of these customers. Fostering loyalty among this group through personalized marketing, rewards programs, or exclusive offers could further enhance their spending.


####  Family members for the customer

In [ ]:
family= data.groupby(["Family_Size"]).size().rename("count").reset_index()

In [ ]:
family.tail()

In [ ]:
bar_chart = px.bar(family, x="Family_Size",y="count", text="count", title= "Family Size of the Customer", hover_name="Family_Size",
       color= "Family_Size",color_discrete_sequence=["orange","red","yellow","brown","gray","blue","purple"],
                template="presentation")
bar_chart.show()

In [ ]:
data.columns

#### Analysis of Anonymised Category for the Customer

In [ ]:
Anonymized_cat= data.groupby(["Category"]).size().rename("count").reset_index()

In [ ]:
Anonymized_cat.head()

In [ ]:
pie_chart= px.pie(Anonymized_cat, names="Category", values="count",
                  title= "Percentage of Anonymized category for the Customer", template="presentation")
pie_chart.show()

### Bivariate Analysis

Bivariate analysis looks at the relationship between two variables and gives us a better understanding of how the two variables interact.

**Visualize the distribution of Numerical features (Age, work_experience  and family size) against the target variable (Segmentation) usign violin plots**



In [ ]:
# Define the features and the target variable
features = ['Age', 'Work_Experience', 'Family_Size']
target_variable = 'Segmentation'
# Set up the subplots
fig, axes = plt.subplots(nrows=1, ncols=len(features), figsize=(18, 6))
# Create violin plots for each feature

for ax, feature in zip(axes, features):
    sns.violinplot(x=target_variable, y=feature, data=data, ax=ax)
    ax.set_title(f'Distribution of {feature} by {target_variable}')
    ax.set_xlabel(target_variable)
    ax.set_ylabel(feature)


**The following observations can be derived from the above plots**

**AGE**
- *Segment D*: Age Range: The distribution is spread from around 20 to 80+ years
- Median Age:The median age seems to be closer to 40:
- Density: There is a slightly higher density around the ages of 30-40 and another around 50-60, indicating two subgroups within this segment.

- Segment A: Age Range: Segment A's age distribution spans from 20 to 80.
- Medain Age:The median age for Segment A is also around 40 years.
- Density: The distribution is slightly more concentrated around the ages of 30-40, meaning more individuals in this segment are younger or middle-aged.

- Segment B: Age Rang: he distribution for Segment B spans from around 20 to 80 years as well. 
- Median Age: The median is around 40 years
- Density: Segment C appears to have a relatively even distribution of ages, with slight concentrations in the middle-age range (30-50).

- Segment C: Age Range: Segment C exhibits a similar age range of 20 to 80 years.
- Median Age: The median is around 40 years.
- Density: Segment C appears to have a relatively even distribution of ages, with slight concentrations in the middle-age range (30-50)


**Work_Experience**

- Key Takeawys: - Low Median Work Experience: Across all segments, the median work experience is consistently low, around 1-2 years. This suggests that the majority of individuals across these segments have relatively little work experience. - Segment D and A: These segments show broader distributions of work experience, with more individuals having moderate to higher levels of experience (up to 4-6 years), compared to Segments B and C - Segment C: This segment has the tightest distribution of work experience, with a strong concentration around low levels of experience (1 year), indicating that most individuals in Segment C are early in their career or have relatively little work experience

**Family_Size**

- Key Takeaways:- Consistent Median Family Size: Across all segments, the median family size is consistently around 2, meaning that the typical family size in all segments is around 2 members. - Distribution and Density: In all segments, family sizes of 1, 2, and 3 are the most frequent, while larger family sizes (above 4) are less common. Segment C seems to have a slightly narrower distribution, with fewer large families. - Segment D: Shows a bit more spread in family size values, with some larger family sizes appearing more frequently compared to other segments. - Segment C: Has a more compact distribution, indicating fewer families with large sizes compared to other segments.



#### Customer Segmentation based on Gender

In [ ]:
gender_based= pd.pivot_table(data, values='ID',index=['Gender'],columns=['Segmentation'],aggfunc=np.count_nonzero)

In [ ]:
gender_based

In [ ]:
fig= px.histogram(data, x="Segmentation", color= "Gender", template="presentation", title="Segmentation based on Gender")
                 
fig.update_layout(bargap=0.1)
fig.show()

####  Segmentation based on Martial Status

In [ ]:
married_based= pd.pivot_table(data, values='ID',index=['Ever_Married'],columns=['Segmentation'],aggfunc=np.count_nonzero)

married_based

In [ ]:
fig= px.histogram(data, x="Ever_Married",color="Segmentation",barmode="group",template="presentation",
           width=1000,height=600, title= "Segmentation based on Marital Status")
fig.update_layout(bargap=0.2)
fig.show()

#### Segmentation based on Graduated

In [ ]:
grad_based= pd.pivot_table(data,values='ID',index=['Graduated'],columns=['Segmentation'],aggfunc=np.count_nonzero)

grad_based

In [ ]:
fig= px.histogram(data, x="Graduated",color="Segmentation",barmode="group",template="presentation",
           width=1000,height=600, title= "Customer Segmentation based on Education Level")
fig.update_layout(bargap=0.2)
fig.show()

#### Segmentation based on Spending score

In [ ]:
spending_based= pd.pivot_table(data, values= 'ID',index=['Spending_Score'],columns=['Segmentation'],aggfunc=np.count_nonzero)

spending_based

In [ ]:
fig= px.bar(data, x="Segmentation",color="Spending_Score", barmode="group",template="presentation",
            title= "Customer Segmentation based on Spending Score")
fig.update_layout(bargap=0.2)
fig.show()

#### Segmentation based on Profession

In [ ]:
prof_based= pd.pivot_table(data, values= 'ID',index=['Profession'],columns=['Segmentation'],aggfunc=np.count_nonzero)

prof_based

In [ ]:
fig= px.bar(data, x="Segmentation",color="Profession", barmode="group",template="presentation",
            title= " Segmentation based on Profession")
fig.update_layout(bargap=0.2)
fig.show()

#### Segmentation based on Work Experience

In [ ]:
work_based= pd.DataFrame(data.groupby('Segmentation')['Work_Experience'].mean())
work_based

In [ ]:
fig= px.bar(work_based, x= work_based['Work_Experience'].index, color= "Work_Experience", template="presentation", title="Segmentation based on Work_Experience")
                 
fig.update_layout(bargap=0.1)
fig.show()

#### Segmentation based o Family Size

In [ ]:
family_based= pd.DataFrame(data.groupby('Segmentation')['Family_Size'].agg(pd.Series.mode))

family_based

In [ ]:
fig= px.bar(family_based, x= family_based['Family_Size'].index, color= "Family_Size", template="presentation", title="Segmentation based on Family Size")
                 
fig.show()

#### Segmentation based on Category

In [ ]:
var_based= pd.pivot_table(data, values= 'ID',index=['Category'],columns=['Segmentation'],aggfunc=np.count_nonzero)

var_based

In [ ]:
fig= px.bar(data, x="Segmentation",color="Category", barmode="group",template="presentation",
            title= " Segmentation based Category")
fig.update_layout(bargap=0.2)
fig.show()

#### Segmentation based on Age

In [ ]:
age_based= pd.DataFrame(data.groupby('Segmentation')['Age'].mean())

age_based

In [ ]:
fig= px.bar(age_based, x= age_based['Age'].index, color= "Age", template="presentation", title="Segmentation based on Age")
                 
fig.show()

In [ ]:
sns.pairplot(data =data[['Age','Graduated','Work_Experience','Family_Size','Segmentation']], hue='Segmentation', palette=["Red","Green","yellow",
                                                                                                     "blue"])
plt.show()

#### Spending score based on Gender

In [ ]:
fig, axes=plt.subplots(1,2)

sns.countplot(data=data, x='Gender',ax= axes[0])

sns.countplot(data=data, x='Gender',hue='Spending_Score',ax=axes[1])
plt.show()

#### Spending Score Based on Profession

In [ ]:
sns.countplot(data=data, x='Profession',hue='Spending_Score')
plt.show()

####  Spending Score based on Family Size

In [ ]:
sns.countplot(data= data, x='Family_Size',hue='Spending_Score')
plt.show()

**Utilized Count plots to visualize the distribution of categorical features and their correlation with Segmentation**

In [ ]:
features = ['Gender', 'Ever_Married', 'Profession', 'Spending_Score', 'Category']
fig, axs = plt.subplots(len(features), 2, figsize=(15, 30))
for i, col in enumerate(features):
    # First count plot
    sns.countplot(x=col, data=data, ax=axs[i, 0])
    axs[i, 0].set_title(f'Count of {col}')
    axs[i, 0].tick_params(axis='x', rotation=40)
    
    # Second count plot with hue
    sns.countplot(x=col, hue='Segmentation', data=data, ax=axs[i, 1])
    axs[i, 1].set_title(f'Count of {col} by Segmentation')
    axs[i, 1].tick_params(axis='x', rotation=40)
plt.tight_layout()
plt.show()

**Here are some of the insights derived from the above plots**

**Gender**
- Male population is slightly higher than female

**Ever_Married**
- Count of married customers are higher 
- Segment D seems to have more number of unmarried customers
- Segment C and B have more number of married customers 


 #### Graduated based on Profession

In [ ]:
sns.countplot(data=data,x='Profession', hue='Graduated')
plt.show()

## Feature Engineering

Feature engineering is the process of transforming raw data into features that are suitable for machine learning models. Its main aim is to improve model accuracy by providing more meaningful and relevant information. This includes feature creation and feature transformation:

-	  Feature creation: is the process of generating new features based on domain know knowledge and helps improve model performance, increase model robustness, improve model interpretability, and increase model flexibility 

-	 Feature transformation: is the process of transforming the features into suitable representations so that the machine learning can learn effectively from the data. Feature transformation improves model performance, increase model robustness, improve computational efficiency and improve model interpretability. This includes :

** Normalization: rescaling the features to have similar range to prevent some feature from dominating others such as 0 and 1 

** Scaling: is used to transform numerical variables to have similar scale with a mean 0 and standard deviation 1

** Encoding:  this is transforming categorical features into numerical representations. Example label encoding, ordinal encoding and one-hot encoding.
 




###  Ordinal Encoding

In [ ]:
 df_customer= pd.DataFrame(data)

In [ ]:
df_customer.head()

In [ ]:
# ordinal Endcoding for spending score

encoder =OrdinalEncoder(categories=[['Low', 'Average','High']])
df_customer['Spending_Score'] = encoder.fit_transform(df_customer[['Spending_Score']])
df_customer.head()

### Label Encoding

In [ ]:
df_customer['Segmentation'].unique()

In [ ]:
# Label Encoding  for the target varialbe 'Segmentation'

df_customer[['Segmentation']]= df_customer[['Segmentation']].apply(LabelEncoder().fit_transform)

df_customer.head()

In [ ]:
# Label encoder on profession  and Category features

df_customer[['Profession']]= df_customer[['Profession']].apply(LabelEncoder().fit_transform)

df_customer[['Category']]= df_customer[['Category']].apply(LabelEncoder().fit_transform)


df_customer.head()

In [ ]:
# conver categorical variables [ Gender, Graduated , Ever_Married ] using predefined mapping to Numerical values 

df_customer['Gender']= df_customer['Gender'].map({'Male':0, 'Female':1})

df_customer['Graduated']= df_customer['Graduated'].map({'No':0, 'Yes':1})

df_customer['Ever_Married']= df_customer['Ever_Married'].map({'No':0, 'Yes':1})





In [ ]:
df_customer.head()

 ### Feature selection

In [ ]:
# let's check the correlation matrix to visualize how much correltion exist between variables

matrix= df_customer.corr().round(2)
sns.heatmap(matrix,annot=True, cmap='vlag')
plt.show()

    



In [ ]:
x= df_customer.drop('Segmentation', axis=1)
y= df_customer['Segmentation']

In [ ]:
# Let’s identify the columns that significantly influence the segmentation column (the target variable) using 
# mutual information tool from scikit learn package. By checking the values, we will drop columns with little 
# to no influence

importances= mutual_info_classif(x,y)
feature_importances= pd.Series(importances, df_customer.columns[0:len(df_customer.columns)-1])
feature_importances.plot(kind='barh',color='teal')
plt.show()

 From the above feature important graph, we learn that gender doe does not have any influence on the target variable so let drop it. We need also to drop the ID columns as it is not relevant to our analysis.
 

###  Splitting the dataset into training and testing 

In [ ]:
x= df_customer.drop(['ID','Segmentation', 'Gender'], axis=1)
y= df_customer['Segmentation']

In [ ]:
x.head()

In [ ]:
y.head()

In [ ]:
# split the data into 80% training and 20% testing with random state of 42 to guarantees reproducibility

x_train,x_test, y_train,y_test= train_test_split(x,y,test_size=0.2, random_state=42)

### Feature Scaling

Many machine learning algorithms like logistic regression, KNN, random forest, etc. require data scaling to produce good results.  For this reason, I will use standard scaler to standardize the features.  standard scaler transforms the distribution of each feature to have mean of zero and standard deviation of one. It standardizes features by subtracting the mean value from the feature and dividing the result by the feature standard deviation. This ensures that all features are on the same scale, preventing any single feature from dominating the learning process due to its large magnitude.

Z=x−μ/σ

were 
- x represents the original feature value, 
- μ is the mean of the feature, 
- σ is the standard deviation, and 
- z is the standardized feature value




In [ ]:
# Apply standardization using Standard Scaler 

scaler = StandardScaler()

x_train= scaler.fit_transform(x_train)

x_test= scaler.transform(x_test)

In [ ]:
print(x_train)

##  Model Development and Prediction

### Logistic Regression

In [ ]:
# Instantiate the model 

logreg= LogisticRegression( max_iter=6000, multi_class='ovr',solver='lbfgs', random_state=42)

# fit the model with data

logreg.fit(x_train,y_train)

y_pred_lr= logreg.predict(x_test)


#### Model Evaluation

In [ ]:

LRACC= accuracy_score(y_pred_lr,y_test)

print(f" The accuracy score for Logistic regression is {(accuracy_score(y_test,y_pred_lr)*100).round(2)} %")


# Classification report

print(classification_report(y_test,y_pred_lr))



#  Confusion Matrix

cm =confusion_matrix(y_test,y_pred_lr)

print(cm)

# visualizing confusion matrix usig heatmap

ConfusionMatrixDisplay(cm).plot()
plt.tight_layout()
plt.title('confusion Matrix')

plt.show()


### K-Nearest Neighbor(KNN)

K-nearest neighbor or K-NN   is used to solve the classification model problem, by creating an imaginary boundary to classify the data.  Lager k means smother curves of separation resulting less complex model whereas smaller K tends to overfit the data and leads to complex modes.


In [ ]:
 # lets find he best value of K uisng cross validation
    
k_values=  [i for i in range(1,50)]  
scores= []
for k in k_values:
    knn= KNeighborsClassifier(n_neighbors=k)
    score= cross_val_score(knn,x,y,cv=5)
    scores.append(np.mean(score))
    
    
sns.lineplot(x= k_values,y= scores, marker ='o')  
plt.xlabel('K Values')
plt.ylabel('Accuracy Score')

In [ ]:
# Instantiate the model 

knn= KNeighborsClassifier(n_neighbors= 21)


# fit the model with the data

knn.fit(x_train, y_train)

y_pred_knn = knn.predict(x_test)

##### Model Evaluation

In [ ]:
accuracy =accuracy_score(y_test, y_pred_knn)


print(f" The accuracy score for K-NN  is {(accuracy_score(y_test,y_pred_knn)*100).round(2)} %")

In [ ]:
# Classification report

print(classification_report(y_test,y_pred_knn))



#  Confusion Matrix

cm =confusion_matrix(y_test,y_pred_knn)

print(cm)

# visualizing confusion matrix usig heatmap

ConfusionMatrixDisplay(cm).plot()
plt.tight_layout()
plt.title('confusion Matrix')

plt.show()


### Support Vector Machine (SVM)

Support vector machine (SVM) is a powerful machine learning algorithm used for linear or nonlinear classification, regression and for outlier detection.  It is adaptable and efficient in many applications as it can handle high dimensional data and nonlinear relationships.  The main aim of the SVM algorithm is to find optimal hyperplane in an N-dimensional space that can separate the data points in different classes in the feature space.

In [ ]:
 # hyper parametre tuning
    
#estimator= SVC()   
#param_grid= {"c":[1,3,5,7,9],"kernel":["linear","polynomial","rbf","sigmoid"]}
#model1= GridSearchCV(estimator, param_grid,cv=5, scoring="accuracy")
#model1.fit(x_train,y_train)
#model1.best_params_
                     

In [ ]:
#Instantiate the model

svm= SVC(C=2, kernel='rbf')
svm.fit(x_train,y_train)

y_pred_svm= svm.predict(x_test)

#### Model Evaluation

In [ ]:
accuracy =accuracy_score(y_test, y_pred_svm)


print(f" The accuracy score for SVM  is {(accuracy_score(y_test,y_pred_svm)*100).round(2)} %")

In [ ]:
# Classification report

print(classification_report(y_test,y_pred_svm))



#  Confusion Matrix

cm =confusion_matrix(y_test,y_pred_svm)

print(cm)

# visualizing confusion matrix usig heatmap

ConfusionMatrixDisplay(cm).plot()
plt.tight_layout()
plt.title('confusion Matrix')

plt.show()


### Decision Tree

decision trees combine multiple points and weigh degrees of uncertainty to determine the best approach to a complex decision.  It allows us to break down information into multiple variables to arrive at a the best single one to the problem. It also helps companies to create informed opinions that facilitate better decision making. 

Random forest algorithm differ from decision trees in their ability to form several decisions in order  to reach  a final  decision, it goes a step  further and don’t rely on a single decision like decision tree. 


#### Hyper parametre Tuning

In [ ]:
   
estimator= DecisionTreeClassifier()
param_grid= {'criterion': ['gini','entropy']}
model=GridSearchCV(estimator,param_grid,cv=5, scoring='accuracy')
model.fit(x_train,y_train)
model.best_params_


In [ ]:
# Instantiate the model

dt= DecisionTreeClassifier(criterion='gini')


# fit the model with  data 

dt.fit(x_train,y_train)
y_pred_dt= dt.predict(x_test)

#### Model Evaluation

In [ ]:
accuracy =accuracy_score(y_test, y_pred_dt)

print(f" The accuracy score for Decision tree  is {(accuracy_score(y_test,y_pred_dt)*100).round(2)} %")


In [ ]:
# Classification report

print(classification_report(y_test,y_pred_dt))



#  Confusion Matrix

cm =confusion_matrix(y_test,y_pred_dt)

print(cm)

# visualizing confusion matrix usig heatmap

ConfusionMatrixDisplay(cm).plot()
plt.tight_layout()
plt.title('confusion Matrix')

plt.show()


### Random Forest

#### Hyper parameter Tuning

In [ ]:
estimator= RandomForestClassifier( random_state=42)
param_grid= {'n_estimators': list(range(1,50))}
model=GridSearchCV(estimator,param_grid,cv=5, scoring='accuracy')
model.fit(x_train,y_train)
model.best_params_


In [ ]:
# Instantiate the model

rf= RandomForestClassifier(n_estimators= 41)


# fit the model with  data 

rf.fit(x_train,y_train)
y_pred_rf= rf.predict(x_test)

#### Model Evaluation

In [ ]:
accuracy =accuracy_score(y_test, y_pred_rf)


print(f" The accuracy score for Random Forest  is {(accuracy_score(y_test,y_pred_rf)*100).round(2)} %\n")




# Classification report

print(classification_report(y_test,y_pred_rf) )



#  Confusion Matrix

cm =confusion_matrix(y_test,y_pred_rf)

print(cm)

# visualizing confusion matrix usig heatmap

ConfusionMatrixDisplay(cm).plot()
plt.tight_layout()
plt.title('confusion Matrix')

plt.show()





#### Summary

 According to the performance metrics  support vector machine achieve the highest accuracy which is 47.64% whereas Decision tree achieve the lowest accuracy which is 37.4%
 
- 	Logistic regression achieved an accuracy of 44.55% 

-   K-Nearest Neighbors (KNN) achieved an accuracy of 46%
- 	Support vector machine (SVM) achieved an accuracy of 47.64%
- 	Decision tree achieved an accuracy of 37.4%
- 	Random Forest achieved an accuracy of 41.75%


**I chose Random Forest for predicting customer behavior despite it accuracy of 41% due to several key points**

 •	**Robustness**: Random Forest is known for its robustness to overfitting, especially when dealing with high-dimensional data. Its ensemble nature helps to reduce variance, making it more reliable in diverse datasets.
 
•	**Feature Importance**: Random Forest provides insights into feature importance, allowing you to understand which variables significantly influence customer behavior. This can be crucial for strategic decision-making.

•	**Handling Missing Data**: Random Forest can handle missing values more effectively than some other algorithms. It can maintain accuracy even when a portion of the data is missing, which is often the case in customer datasets.

•	**Lower Sensitivity to Outliers**: Random Forest is less sensitive to outliers compared to models like Logistic Regression and Decision Trees, making it a better choice for real-world datasets that may contain noise.

•	**Ensemble Learning**: The ensemble approach of Random Forest combines the predictions of multiple decision trees, leading to better generalization on unseen data compared to single models.

•	**Tuning Potential**: While the initial accuracy might not be the highest, Random Forest has various hyperparameters that can be tuned (e.g., number of trees, depth of trees) to potentially improve performance further.



**Hyperparameter Tuning - GridSearchCV** 

First, let’s use GridSearchCV to obtain the best parameters for the model. For that, we will pass RandomFoestClassifier() instance to the model and then fit the GridSearchCV using the training data to find the best parameters.

In [ ]:
rf = RandomForestClassifier()
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}




In [ ]:
# set up the gird search 
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy')

# fit the model
grid_search.fit(x_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_


In [ ]:
print("Best Hyperparameters:", best_params)

In [ ]:
# Make predictions on the test set

y_pred = best_estimator.predict(x_test)


In [ ]:
# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Classification Report
print(classification_report(y_test, y_pred))

**Interpretation of the performance metrics** 

**Confusion Matrix**

1	**True Positives (TP)**

• Segment A: 252 instances were correctly predicted as A.

• Segment B: 130 instances were correctly predicted as B.

• Segment C: 256 instances were correctly predicted as C.

• Segment D: 396 instances were correctly predicted as D.

2. **False Negatives (FN)**

•	For A: 88 instances were misclassified as B, 95 as C, and 149 as D. Total FN for A = 88 + 95 + 149 = 332.

•	For B: 151 instances were misclassified as A, 126 as C, and 83 as D. Total FN for B = 151 + 126 + 83 = 360.

•	For C: 75 instances were misclassified as A, 81 as B, and 60 as D. Total FN for C = 75 + 81 + 60 = 216.

•	For D: 130 instances were misclassified as A, 39 as B, and 28 as C. Total FN for D = 130 + 39 + 28 = 197

3.**False Positives (FP)**

•	For A: 151 instances of B, 75 instances of C, and 130 instances of D were incorrectly predicted as A. Total FP for A = 151 + 75 + 130 = 356.
•	For B: 88 instances of A, 81 instances of C, and 39 instances of D were incorrectly predicted as B. Total FP for B = 88 + 81 + 39 = 208.
•	For C: 95 instances of A, 126 instances of B, and 28 instances of D were incorrectly predicted as C. Total FP for C = 95 + 126 + 28 = 249.
•	For D: 149 instances of A, 83 instances of B, and 60 instances of C were incorrectly predicted as D. Total FP for D = 149 + 83 + 60 = 292.


**Summary of Findings**

•**High True Positives**: Segments C and D show strong identification with 256 and 396 correct predictions, indicating good model performance for these segments.

•**False Negatives**: Segments A and B have substantial false negatives, particularly A with 332 misclassified instances, suggesting that the model struggles with these segments.

•**False Positives**: The model also misclassifies instances across segments, notably segment A having 356 instances incorrectly predicted from other segments.




**Recommendations for Improvement**

1.**Feature Engineering**: Examine and possibly refine features to improve model performance, particularly for segments A and B.

2.**Hyperparameter Tuning**: Explore different hyperparameter settings for the Random Forest model to find an optimal configuration.

3.**Class Imbalance**: Check if any segments are underrepresented and consider techniques like oversampling or under sampling
